# Rocket Problem with Manual Dynamics

Here, we solve a rocket optimal control problem: the same one previously posed in the [problem scaling tutorial](../../01%20-%20Optimization%20and%20Math/07%20-%20Problem%20Scaling.ipynb), and originally formulated on [the CasADi blog](https://web.casadi.org/blog/nlp-scaling/). In that tutorial section, we merely used this problem as an example to illustrate the importance of problem scaling.

Here, we dive deeper and show how you would set up such a problem.

The imports and initial problem setup are straightforward:

In [3]:
import aerosandbox as asb
import aerosandbox.numpy as np

### Environment
opti = asb.Opti()

### Time discretization
N = 100  # Number of discretization points
time_final = 100  # seconds
time = np.linspace(0, time_final, N)

### Constants
mass_initial = 500e3  # Initial mass, 500 metric tons
y_final = 100e3  # Final altitude, 100 km
g = 9.81  # Gravity, m/s^2
alpha = 1 / (300 * g)  # kg/(N*s), Inverse of specific impulse, basically - don't worry about this

### Variables
y = opti.variable(init_guess=np.linspace(0, y_final, N))  # Altitude
velocity = opti.variable(init_guess=y_final / time_final, n_vars=N)  # Velocity
mass = opti.variable(init_guess=mass_initial, n_vars=N)  # Mass
u = opti.variable(init_guess=g * mass_initial, n_vars=N)  # Control vector

Here's where it gets interesting. We can implement the dynamics as follows, where we *explicitly* write out the relationship between variables using constraints.

This is the best approach if you want the most fine-grained control over exactly what is happening in the integrator:

In [4]:
### Dynamics (implemented manually for now, we'll show you more sophisticated ways to do this in the Trajectory
# Optimization part of the tutorial later on)
opti.subject_to([  # Forward Euler, implemented manually for now
    np.diff(y) == velocity[:-1] * np.diff(time),
    np.diff(velocity) == (u[:-1] / mass[:-1] - g) * np.diff(time),
    np.diff(mass) == (-alpha * u[:-1]) * np.diff(time)
])

[MX(fabs(opti1_lam_g_1)), MX(fabs(opti1_lam_g_2)), MX(fabs(opti1_lam_g_3))]

We the continue to write out the problem, and solve it.

In [5]:
### Boundary conditions
opti.subject_to([
    y[0] == 0,
    velocity[0] == 0,
    mass[0] == mass_initial,
    y[-1] == y_final
])

### Path constraints
opti.subject_to([
    mass >= 0,
    u >= 0
])

### Objective
opti.minimize(-mass[-1])  # Maximize the final mass == minimize fuel expenditure

### Solve
sol = opti.solve(verbose=False)
print(f"Solved in {sol.stats()['iter_count']} iterations.")

Solved in 13 iterations.


Okay, let's go back to that dynamics section of this problem - this part:

```python
opti.subject_to([  # Forward Euler, implemented manually for now
    np.diff(y) == velocity[:-1] * np.diff(time),
    np.diff(velocity) == (u[:-1] / mass[:-1] - g) * np.diff(time),
    np.diff(mass) == (-alpha * u[:-1]) * np.diff(time)
])
```

We are effectively defining derivatives here. For example, the first constraint basically implements a discrete version of the definition:

$$ \frac{dy}{dt} = v \implies \Delta y = \int_{t_1}^{t_2} v(t)\ dt$$

by approximating it via the forward-Euler discretization:

$$ \Delta y \approx v(t_1) \cdot \Delta t$$

There are a couple of alternative ways that we could write this out, which are more abstracted and simple. For example, here's an alternate way of implementing the dynamics:

In [ ]:
opti.constrain_derivative(
    derivative=velocity,
    variable=y,
    with_respect_to=time,
)
opti.constrain_derivative(
    derivative=u / mass - g,
    variable=velocity,
    with_respect_to=time,
)
opti.constrain_derivative(
    derivative=-alpha * u,
    variable=mass,
    with_respect_to=time,
)

Personally, I think this is a lot more readable than the more-mathematical `opti.subject_to()` implementation above - but that's open to opinion.